### Project Description

#### Project Overview:
You have been hired as a Data Engineer by a research organization, tasked with developing an automated Python solution to extract, transform, and load (ETL) data on the **top 10 largest banks in the world** based on market capitalization. The data needs to be transformed to include market capitalization in various currencies (GBP, EUR, INR) using exchange rate information provided in a separate CSV file. This system will be used to generate reports for the organization on a quarterly basis.

The final output will include:
- A **CSV file** containing the market capitalization of the top 10 banks in USD, GBP, EUR, and INR.
- A **database** storing the same information for easier querying and analysis.

The process must be fully automated so that the same script can be run every financial quarter to ensure up-to-date information is compiled.

### Project Details:

- **Code Name**: `banks_project.py`
- **Data Source URL**: [Top 10 Largest Banks by Market Capitalization](https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks)
- **Exchange Rate CSV Path**: [Exchange Rate Data](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv)
- **Initial Table Attributes (Upon Extraction)**: `Name`, `MC_USD_Billion`
- **Final Table Attributes (After Transformation)**: `Name`, `MC_USD_Billion`, `MC_GBP_Billion`, `MC_EUR_Billion`, `MC_INR_Billion`
- **Output CSV File Path**: `./Largest_banks_data.csv`
- **Database Name**: `Banks.db`
- **Database Table Name**: `Largest_banks`
- **Log File Name**: `code_log.txt`

### Project Tasks:

#### **Task 1: Log Progress**  
- **Objective**: Write a function `log_progress()` to track and log the progress of the script at different stages.
- **Details**: Use logging to create entries in `code_log.txt` at key points in the process, such as after extraction, transformation, loading to CSV, and loading to the database.

#### **Task 2: Data Extraction**  
- **Objective**: Extract the list of the top 10 largest banks by market capitalization from the provided URL.
- **Details**:
  1. Inspect the webpage to identify the position and structure of the table containing the required data.
  2. Write a function `extract()` to scrape the data and load it into a DataFrame.
  3. Execute the extraction and verify that the DataFrame correctly contains the list of banks with their market capitalization in USD.

#### **Task 3: Data Transformation**  
- **Objective**: Transform the extracted data by converting market capitalization from USD to GBP, EUR, and INR using the exchange rate data provided in the CSV file.
- **Details**:
  1. Write a function `transform()` that adds new columns for the market capitalization in GBP, EUR, and INR based on the provided exchange rates.
  2. Ensure that all values are rounded to two decimal places.
  3. Verify that the transformation correctly computes the market capitalization in all four currencies.

#### **Task 4: Load Data to CSV**  
- **Objective**: Save the transformed DataFrame to a CSV file.
- **Details**:
  - Write a function `load_to_csv()` to write the transformed data to `Largest_banks_data.csv`.
  - Verify that the output CSV file contains the correct data.

#### **Task 5: Load Data to SQL Database**  
- **Objective**: Load the transformed data into an SQLite database.
- **Details**:
  1. Write a function `load_to_db()` that creates a table in the database and inserts the transformed data.
  2. Ensure the table is created if it does not already exist and that the data is correctly inserted.
  3. Verify that the data is loaded into the `Largest_banks` table in the `Banks.db` database.

#### **Task 6: Query the Database**  
- **Objective**: Run SQL queries on the database to retrieve and verify the data.
- **Details**:
  1. Write a function `run_query()` to execute a SQL query that retrieves banks with market capitalization greater than 100 billion USD.
  2. Display the results of the query and ensure they are correct.

#### **Task 7: Verify Log Entries**  
- **Objective**: Verify that the `code_log.txt` file contains log entries for all the key stages.
- **Details**: After completing all tasks, check the log file to ensure that the process is fully logged, including the start and end of the ETL process, and the completion of each task.

### Expected Outcomes:
1. **CSV Output**: A file `Largest_banks_data.csv` containing the list of top 10 largest banks and their market capitalization in USD, GBP, EUR, and INR.
2. **Database Output**: An SQLite database file `Banks.db` with a table `Largest_banks` storing the same data.
3. **SQL Query Results**: A query output showing the banks with market capitalization greater than 100 billion USD.
4. **Log File**: A log file `code_log.txt` capturing the execution progress with timestamps at each stage.

### Tools and Technologies:
- **Python**: For implementing the ETL process.
- **Libraries**:
  - `requests` and `BeautifulSoup` for web scraping.
  - `pandas` for data manipulation and storage in CSV format.
  - `sqlite3` for interacting with the SQLite database.
  - `logging` for tracking progress and errors during the execution.
  
### Conclusion:
This project will automate the process of gathering, transforming, and storing data on the top 10 largest banks in the world by market capitalization, enabling the research organization to generate up-to-date reports every quarter. The code will fetch the latest data from the internet, apply necessary currency conversions, and store the data in both CSV and database formats for easy querying and analysis. The progress will be logged to ensure traceability and error handling during the process.

In [ ]:
import requests
import pandas as pd
import sqlite3
import logging
from bs4 import BeautifulSoup

# Setup logging configuration
logging.basicConfig(filename='code_log.txt', level=logging.INFO, format='%(asctime)s - %(message)s')

def log_progress(message):
    """This function logs the mentioned message of a given stage of the code execution to a log file."""
    logging.info(message)

def extract(url, table_attribs):
    """This function extracts the required information from the website and saves it to a DataFrame."""
    log_progress("Extracting data from URL")

    # Request the content from the URL
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')

    # Locate the table with market capitalization information
    table = soup.find('table', {'class': 'wikitable'})

    # Extract the header and rows of the table
    headers = [header.text.strip() for header in table.find_all('th')]
    rows = table.find_all('tr')[1:]  # Skip the header row

    data = []
    for row in rows:
        columns = row.find_all('td')
        if len(columns) > 1:  # Avoid empty rows
            name = columns[1].text.strip()
            market_cap_usd = columns[2].text.strip().replace(',', '')
            data.append([name, market_cap_usd])

    # Convert to DataFrame
    df = pd.DataFrame(data, columns=table_attribs)

    log_progress("Data extracted successfully")
    return df

def transform(df, csv_path):
    """This function transforms the Market Cap values from USD to GBP, EUR, INR based on exchange rates."""
    log_progress("Transforming data to different currencies")

    # Read the exchange rate CSV file
    exchange_rates = pd.read_csv(csv_path)

    # Convert Market Cap to numeric (USD)
    df['MC_USD_Billion'] = pd.to_numeric(df['MC_USD_Billion'], errors='coerce')

    # Get exchange rates from the CSV file
    usd_to_gbp = exchange_rates.loc[exchange_rates['Currency'] == 'GBP', 'Rate'].values[0]
    usd_to_eur = exchange_rates.loc[exchange_rates['Currency'] == 'EUR', 'Rate'].values[0]
    usd_to_inr = exchange_rates.loc[exchange_rates['Currency'] == 'INR', 'Rate'].values[0]

    # Add the transformed columns to the DataFrame
    df['MC_GBP_Billion'] = df['MC_USD_Billion'] * usd_to_gbp
    df['MC_EUR_Billion'] = df['MC_USD_Billion'] * usd_to_eur
    df['MC_INR_Billion'] = df['MC_USD_Billion'] * usd_to_inr

    # Round the values to 2 decimal places
    df = df.round({'MC_USD_Billion': 2, 'MC_GBP_Billion': 2, 'MC_EUR_Billion': 2, 'MC_INR_Billion': 2})

    log_progress("Data transformed successfully")
    return df

def load_to_csv(df, output_path):
    """This function saves the final DataFrame as a CSV file."""
    log_progress("Loading data to CSV")
    df.to_csv(output_path, index=False)
    log_progress(f"Data saved to {output_path} successfully")

def load_to_db(df, sql_connection, table_name):
    """This function saves the final DataFrame to a database table."""
    log_progress(f"Loading data to database table {table_name}")

    # Create a table if it doesn't exist
    cursor = sql_connection.cursor()
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Name TEXT,
            MC_USD_Billion REAL,
            MC_GBP_Billion REAL,
            MC_EUR_Billion REAL,
            MC_INR_Billion REAL
        )
    """)

    # Insert data into the table
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    log_progress("Data loaded to database successfully")

def run_query(query_statement, sql_connection):
    """This function runs a query on the database table and prints the output."""
    log_progress(f"Running query: {query_statement}")
    cursor = sql_connection.cursor()
    cursor.execute(query_statement)
    result = cursor.fetchall()

    for row in result:
        print(row)
    log_progress("Query executed successfully")

# Main Program to execute the tasks

if __name__ == '__main__':
    # Task 1: Log the start of the process
    log_progress("ETL Process Started")

    # Task 2: Extract data from URL
    url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
    table_attribs = ['Name', 'MC_USD_Billion']
    df = extract(url, table_attribs)

    # Task 3: Transform data using exchange rates from CSV
    exchange_rate_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'
    df_transformed = transform(df, exchange_rate_path)

    # Task 4: Load transformed data to CSV
    output_csv_path = './Largest_banks_data.csv'
    load_to_csv(df_transformed, output_csv_path)

    # Task 5: Load transformed data to the database
    db_connection = sqlite3.connect('Banks.db')
    table_name = 'Largest_banks'
    load_to_db(df_transformed, db_connection, table_name)

    # Task 6: Run queries on the database
    query = "SELECT * FROM Largest_banks WHERE MC_USD_Billion > 100"
    run_query(query, db_connection)

    # Task 7: Verify that log entries are complete
    log_progress("ETL Process Completed")

    # Close database connection
    db_connection.close()


('JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('Bank of America', 231.52, 185.22, 215.31, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('Bank of China', 136.81, 109.45, 127.23, 11348.39)
